In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
from allennlp.common.util import ensure_list
from allennlp.data.data_loaders import SimpleDataLoader
from allennlp.data.batch import Batch
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.token_embedders import Embedding

import torch.nn.functional as F
from torch import optim

In [4]:
import sys
sys.path.insert(0,'/Users/puneet/Projects/pytorch/practice/book_nlp')

In [5]:
from nlp.readers.word2vec_reader import SkipGramReader
from nlp.models.skip_gram import SkipGramNegativeSamplingModel,SkipGramModel

In [6]:
reader = SkipGramReader(load_from_pkl='../junks/word_table.pkl')
instances = ensure_list(reader.read('../data/text8'))

Your label namespace was 'tags_in'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.


count value  95
count value  90
count value  85
count value  80
count value  75
count value  70
count value  65
count value  60
count value  55
count value  50
count value  45
count value  40
count value  35
count value  30
count value  25
count value  20
count value  15
count value  10
count value  5
count value  0


In [7]:
vocab = Vocabulary.from_instances(instances)
dls = SimpleDataLoader(instances, batch_size=4)

dls.index_with(vocab)

In [8]:
next(iter(dls))

{'tokens_in': tensor([  3,   3, 531, 531]),
 'tokens_out': tensor([531,  44,   3,  44]),
 'negatives': tensor([[3633,  235, 3634, 1469, 1470],
         [  98, 3635, 3636, 3637,  107],
         [3638,  947, 3639, 3640, 1471],
         [ 567, 3641, 3642, 3643, 3644]])}

In [38]:
src_embedding = Embedding(embedding_dim=300,vocab_namespace='tags_in',vocab=vocab, sparse=True)
dst_embedding = Embedding(embedding_dim=300,vocab_namespace='tags_in', vocab=vocab, sparse=True)

model = SkipGramModel(vocab, src_embedding, dst_embedding)

In [39]:
bs = next(iter(dls))

In [40]:
bs

{'tokens_in': tensor([  3,   3, 531, 531]),
 'tokens_out': tensor([531,  44,   3,  44]),
 'negatives': tensor([[3633,  235, 3634, 1469, 1470],
         [  98, 3635, 3636, 3637,  107],
         [3638,  947, 3639, 3640, 1471],
         [ 567, 3641, 3642, 3643, 3644]])}

In [41]:
optimizer = optim.Adam(model.parameters())

optimizer.zero_grad()
loss = model(**bs)
loss['loss'].backward()

> /Users/puneet/Projects/pytorch/practice/book_nlp/nlp/models/skip_gram.py(54)forward()
     52         pdb.set_trace()
     53         # retain_grad because its not happening correctly
---> 54         embedded_in.retain_grad()
     55         embedded_out.retain_grad()
     56         embedded_neg.retain_grad()

ipdb> c


In [42]:
loss

{'loss': tensor(10.5358, grad_fn=<NegBackward>)}

In [45]:
model.embedding_out.weight

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [113]:
src= torch.nn.Embedding(1000,300)
trg = torch.nn.Embedding(1000,300)

In [125]:
src1 = src.weight.data[4:6,:]
trg1 = trg.weight.data[10:12,:]
src1.requires_grad_(True)
trg1.requires_grad_(True)
loss = F.logsigmoid(torch.mul(src1,trg1).sum(dim=1)).sum()

In [126]:
loss.backward()

In [62]:
src = torch.randn((2,300),requires_grad=True)
trg = torch.randn((2,300), requires_grad=True)

In [92]:
model.embedding_in.weight.retain_grad()

In [106]:
tsrc = torch.tensor([2,4])
ttrg = torch.tensor([10,40])
src = model.embedding_in(tsrc)
trg = model.embedding_in(ttrg)
#src.retain_grad()
#trg.retain_grad()

In [107]:
loss = F.logsigmoid(torch.mul(src,trg).sum(dim=1)).sum()

In [103]:
loss.backward()

In [12]:

for batch in dls:
    optim.zero_grad()
    loss = model(**batch)
    loss['loss'].backward()
    optimizer.step()

AttributeError: module 'torch.optim' has no attribute 'zero_grad'

In [128]:
input = torch.tensor([[1,2,4,5],[4,3,2,9]])

In [129]:
embedding_matrix = torch.rand(10, 3)

In [130]:
input.shape

torch.Size([2, 4])

In [131]:
F.embedding(input, embedding_matrix).shape

torch.Size([2, 4, 3])

In [132]:
weights = torch.rand(10, 3)

In [133]:
weights[0, :].zero_()

tensor([0., 0., 0.])

In [134]:
embedding_matrix = weights

In [135]:
input = torch.tensor([[0,2,0,5]])

In [137]:
F.embedding(input, embedding_matrix, padding_idx=0)

tensor([[[0.0000, 0.0000, 0.0000],
         [0.9902, 0.4637, 0.8755],
         [0.0000, 0.0000, 0.0000],
         [0.2321, 0.1385, 0.0298]]])